In [20]:
# import
import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from ..utility.lib import probe_to_gene


from utility.get_data import load_rdata



ImportError: attempted relative import with no known parent package

In [2]:
!pwd

/home/hugo/code/ER_status_prediction/notebook


We are going to train a machine learning model to classify Breast cancer ER (Estrogen receptors) positive and 
negative based on transcriptomic data 

This dataset was originally found [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=gse7390)


In [ ]:
# dowload affymetrix dataset in data folder
! script/get_affy_data.sh

In [ ]:
# load .rDATA
df, annot, demo = load_rdata()

In [ ]:
print(df.shape) # extract gene expression
print(annot.shape) # extract gene information/description
print(demo.shape) # extract clinical data including ER status

In [ ]:
demo.head()

In [ ]:
df.head()

In [ ]:
annot.head()

In [ ]:
gene_list = annot["HUGO.gene.symbol"].tolist()
print(f"{len(gene_list)} total genes")

# housekeeping gene use for control
housekeeping_gene = [colname for colname in list(df.columns) if str(colname).startswith('AFFX')]
print(f"{len(housekeeping_gene)} housekeeping gene")

unknow_gene = [gene for gene in gene_list if str(gene) == 'nan']
print(f"{len(unknow_gene)} unknow_gene")

# remove thoses gene from df
df.drop(housekeeping_gene, inplace=True, axis=1)

In [ ]:
# saved dataframe
df_er = pd.concat([df, demo['er']], axis=1)
df_er.to_csv("data/gene_expression_er.csv", index=False)
annot.to_csv("data/annot.csv", index=False)
df_er

We have 22216 gene including ER cancer status for the 198 patients. Now, lets look at the distribution of patient 
without and ER cancer.

In [ ]:
tumors_type = {'0': 'negative', '1': 'positive'}
labels, counts = np.unique(demo['er'], return_counts=True)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# plot frequency
ax[0].bar(labels[0], counts[0], label=tumors_type[str(labels[0])])
ax[0].bar(labels[1], counts[1], label=tumors_type[str(labels[1])])
ax[0].set_ylabel('Number of patients')
ax[0].axes.get_xaxis().set_ticks([])
ax[0].set_ylim([0, 200])

for i in ax[0].patches:
    height = i.get_height()
    ax[0].text((i.get_x() + i.get_width()/2.0), height + 20 , str(i.get_height()), ha='center',
            fontsize=12, fontweight='bold', color='grey')

#create pie chart
colors = sns.color_palette('pastel')[0:2]
ax[1].pie(counts, labels = tumors_type.values(), autopct='%.0f%%')
ax[1].legend(bbox_to_anchor=(1.9, 1), loc='upper right', borderaxespad=0)

fig.suptitle("ER type distribution", fontsize=16, y=1.05)
plt.subplots_adjust(wspace=0.4)
plt.show()

We have an unbalance dataset (about 1:2 ration) with more ER positive patients. Now, we going to see if we can
cluster this 2 groups using gene expression by clustering. We used PCA and T-SNE

In [ ]:
# standardize the data for dimensionality reduction 
scaler = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
y = demo['er']

In [ ]:
# PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
Df_pca = pd.DataFrame(data = principalComponents, columns = ['PCA1', 'PCA2'])

# Tsne
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(x)
Df_tsne = pd.DataFrame(data = tsne_results, columns = ['tsne-2d-one', 'tsne-2d-two'])

finalDf = pd.concat([Df_pca, Df_tsne], axis=1)
finalDf.index = df.index
finalDf['er'] = y.values
finalDf

In [ ]:
# lets plot PCA and T-SNE variable accoring to ER groups

plt.figure(figsize=(16,8))

tumors_type = {'0': 'negative', '1': 'positive'}

color_dict = {1: '#1f77b4', 2: '#ff7f0e', 3: '#2ca02c'}
color_dict = {1: 'b', 2: 'g', 3: 'r'}

ax1 = plt.subplot(1, 2, 1)
ax1.set_xlabel('Principal Component 1', fontsize = 15)
ax1.set_ylabel('Principal Component 2', fontsize = 15)
ax1.set_title('2 Component PCA', fontsize = 20)

sns.scatterplot(
    x="PCA1", y="PCA2",
    hue="er",
    data=finalDf,
    #palette='Set2', 
    alpha=0.8,
    legend=None,
    ax=ax1)

ax2 = plt.subplot(1, 2, 2)
ax2.set_xlabel('T-SNE-1', fontsize = 15)
ax2.set_ylabel('T-SNE-2', fontsize = 15)
ax2.set_title('T-sne', fontsize = 20)

sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="er",
    data=finalDf,
    #palette='Set2', 
    alpha=0.8,
    ax=ax2)


handles, labels  =  ax2.get_legend_handles_labels()
tumors_labels = [tumors_type[label] for label in labels]

ax2.legend(loc='upper right', bbox_to_anchor=(0.5, -0.5), ncol=1)
ax2.legend(handles, tumors_labels)

plt.show()



We can see the negative and positve ER patient group seems quite easily notably on PCA. 

Now, we are going to select feature to then train a machine learning models to classify ER status on er_prediction.ipynb